# Importação e Tratamento de Dados

In [17]:
import pandas as pd
import numpy as np
from sklearn_helper import preparar_dados_impacto

In [18]:
taxa_de_amostragem_hz = 208

# Configuração da função find_peaks
coluna_picos = 'imu1accz'
espacamento_entre_picos_s = 8 * taxa_de_amostragem_hz
altura_minima_pico_metalico = 5
altura_minima_pico_amortecido = 5

# Configuração da fatia de dados a ser recortada em cada pico
comprimento_da_leitura_s = int(1 * taxa_de_amostragem_hz)
offset_antes_do_pico_s = int(.1 * taxa_de_amostragem_hz)

colunas_a_remover = ['time', 'dac1', 'dac2', 'dac3', 'dac4', 'log']

In [19]:
altura_minima = [altura_minima_pico_metalico, altura_minima_pico_amortecido]

In [20]:
dados_crus_gabriel = {  'metalico': './Dados_Gabriel/Dados_Impacto_Metalico.feather', 
                        'amortecido': './Dados_Gabriel/Dados_Impacto_Amortecido.feather'}

dfs = []
for i, dados in enumerate(dados_crus_gabriel.values()):
    df = pd.read_feather(dados).drop(colunas_a_remover, axis=1)
    df = preparar_dados_impacto(df, coluna_picos, comprimento_da_leitura_s, offset_antes_do_pico_s, altura_minima[i], espacamento_entre_picos_s)
    df['label'] = i
    dfs.append(df)
del df

df_preparado_gabriel = pd.concat(dfs).dropna()
del dfs

df_preparado_gabriel

,imu1accx_0,imu1accy_0,imu1accz_0,imu1gyrox_0,imu1gyroy_0,imu1gyroz_0,imu2accx_0,imu2accy_0,imu2accz_0,imu2gyrox_0,...,imu1gyrox_227,imu1gyroy_227,imu1gyroz_227,imu2accx_227,imu2accy_227,imu2accz_227,imu2gyrox_227,imu2gyroy_227,imu2gyroz_227,label
0,-1.276109,19.478028,0.714669,2.246857,-3.631592,-3.696442,-1.819593,19.391836,0.915782,2.529144,...,-1.071930,-3.791809,-3.738403,-1.738190,19.139248,-5.266047,-4.230499,-7.247925,-2.262115,0
1,-1.262941,19.480422,0.756568,2.151489,-3.593445,-3.719330,-1.845929,19.354726,1.072602,2.300262,...,0.610352,-3.765106,-3.578186,-1.770512,19.592949,1.713051,0.663757,-6.900787,-2.468109,0
2,-1.264138,19.476831,0.695516,2.254486,-3.612518,-3.738403,-1.829170,19.389442,0.824802,2.464294,...,2.464294,-3.887177,-3.726959,-2.064999,19.068619,-0.500388,-0.656128,-6.965637,-2.399445,0
3,-1.260547,19.494787,0.749385,2.449036,-3.681183,-3.692627,-1.816002,19.399019,0.987608,2.769470,...,3.822327,-3.726959,-3.700256,-2.085350,19.796456,-1.892616,4.585266,-6.637573,-2.437592,0
4,-1.285686,19.505561,0.769736,2.372742,-3.517151,-3.608704,-1.807622,19.383457,1.128866,2.784729,...,0.251770,-3.627777,-3.688812,-2.103306,18.825608,-2.163161,-2.956390,-7.122040,-2.307892,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36,-1.265336,19.505561,0.675165,2.223969,-3.734589,-3.765106,-1.862689,19.348741,0.976834,2.342224,...,12.615204,-3.650665,-3.856659,-1.875857,19.521123,-1.449689,15.163422,-5.825043,-2.861023,1
37,-1.265336,19.512744,0.784101,2.407074,-3.822327,-3.787994,-1.856703,19.361909,1.191115,2.407074,...,6.237030,-3.856659,-3.749847,-1.296460,17.858350,-6.609194,3.097534,-6.816864,-2.452850,1
38,-1.261744,19.524715,0.804452,2.838135,-3.849030,-3.822327,-1.830367,19.337967,1.182736,3.105164,...,-2.105713,-4.329681,-3.810883,-2.243367,18.156428,-8.462306,-6.023407,-7.534027,-2.269745,1
39,-1.254562,19.499575,0.746991,2.582550,-3.822327,-3.826141,-1.839944,19.354726,1.250970,2.811432,...,4.138947,-3.677368,-3.822327,-1.674744,19.505561,-0.761356,5.176544,-6.721497,-2.544403,1


In [21]:
dados_crus_joao = { 'metalico': ['./Dados_Joao/Impacto_Metalico_J1.feather', './Dados_Joao/Impacto_Metalico_J2.feather'], 
                    'amortecido': ['./Dados_Joao/Impacto_Amortecido_J1.feather', './Dados_Joao/Impacto_Amortecido_J2.feather']}
dfs = []
for i, tipo_impacto in enumerate(dados_crus_joao.values()):
    for dados in tipo_impacto:
            df = pd.read_feather(dados).drop(colunas_a_remover, axis=1)
            df = preparar_dados_impacto(df, coluna_picos, comprimento_da_leitura_s, offset_antes_do_pico_s, altura_minima[i], espacamento_entre_picos_s)
            df['label'] = i
            dfs.append(df)
del df

df_preparado_joao = pd.concat(dfs).dropna()
del dfs

df_preparado_joao

,imu1accx_0,imu1accy_0,imu1accz_0,imu1gyrox_0,imu1gyroy_0,imu1gyroz_0,imu2accx_0,imu2accy_0,imu2accz_0,imu2gyrox_0,...,imu1gyrox_227,imu1gyroy_227,imu1gyroz_227,imu2accx_227,imu2accy_227,imu2accz_227,imu2gyrox_227,imu2gyroy_227,imu2gyroz_227,label
0,0.859518,19.515138,0.695516,1.766205,-3.540039,-4.333496,-1.679532,19.364303,0.612916,2.456665,...,4.428864,-3.654480,-4.287720,-1.642422,19.442115,1.601721,5.786896,-6.446838,-2.552032,0
1,0.875081,19.518729,0.773327,2.265930,-3.643036,-4.386902,-1.679532,19.399019,0.991199,3.101349,...,2.582550,-3.337860,-4.257202,-2.191892,18.618509,-1.836353,3.025055,-6.435394,-2.422333,0
2,0.878672,19.561825,0.792481,1.380920,-3.620148,-4.455566,-1.648408,19.403807,1.120486,2.067566,...,4.093170,-3.978729,-4.447937,-1.769315,19.674352,3.469198,6.675720,-5.828857,-2.452850,0
3,0.894234,19.553445,0.699107,1.731873,-3.646851,-4.493713,-1.660379,19.405004,0.577003,2.090454,...,1.712799,-3.547668,-4.325867,-1.796848,18.972851,0.052672,2.857208,-6.504059,-2.433777,0
4,0.894234,19.531897,0.804452,2.338409,-3.608704,-4.539490,-1.705869,19.396625,1.176750,3.013611,...,3.582001,-3.803253,-4.428864,-1.738190,19.050663,-1.551443,6.191254,-6.336212,-2.456665,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41,0.879869,19.606117,0.538695,2.185822,-4.207611,-4.436493,-1.788469,19.372683,0.597353,2.891541,...,8.102417,-4.795074,-4.676819,-1.696292,19.320010,-4.583699,12.725830,-6.240845,-2.872467,1
42,0.887052,19.597738,0.635661,1.964569,-4.138947,-4.543304,-1.766921,19.348741,1.211466,2.380371,...,-1.644135,-3.955841,-4.653931,-1.264138,19.425355,-3.442862,-4.035950,-7.827759,-2.449036,1
43,0.859518,19.589358,0.549469,2.170563,-4.215240,-4.650116,-1.790863,19.348741,0.652420,2.677917,...,7.251740,-4.634857,-4.451752,-1.848324,18.938135,-1.472434,8.739471,-6.885529,-2.788544,1
44,0.863110,19.583372,0.579397,1.792908,-3.967285,-4.447937,-1.760935,19.346347,0.720655,2.258301,...,-1.857758,-3.963470,-4.722595,-1.668759,19.218257,-2.339135,-0.911713,-7.133484,-2.452850,1


In [22]:
df_preparado_gabriel_joao = pd.concat([df_preparado_gabriel, df_preparado_joao])
df_preparado_gabriel_joao


,imu1accx_0,imu1accy_0,imu1accz_0,imu1gyrox_0,imu1gyroy_0,imu1gyroz_0,imu2accx_0,imu2accy_0,imu2accz_0,imu2gyrox_0,...,imu1gyrox_227,imu1gyroy_227,imu1gyroz_227,imu2accx_227,imu2accy_227,imu2accz_227,imu2gyrox_227,imu2gyroy_227,imu2gyroz_227,label
0,-1.276109,19.478028,0.714669,2.246857,-3.631592,-3.696442,-1.819593,19.391836,0.915782,2.529144,...,-1.071930,-3.791809,-3.738403,-1.738190,19.139248,-5.266047,-4.230499,-7.247925,-2.262115,0
1,-1.262941,19.480422,0.756568,2.151489,-3.593445,-3.719330,-1.845929,19.354726,1.072602,2.300262,...,0.610352,-3.765106,-3.578186,-1.770512,19.592949,1.713051,0.663757,-6.900787,-2.468109,0
2,-1.264138,19.476831,0.695516,2.254486,-3.612518,-3.738403,-1.829170,19.389442,0.824802,2.464294,...,2.464294,-3.887177,-3.726959,-2.064999,19.068619,-0.500388,-0.656128,-6.965637,-2.399445,0
3,-1.260547,19.494787,0.749385,2.449036,-3.681183,-3.692627,-1.816002,19.399019,0.987608,2.769470,...,3.822327,-3.726959,-3.700256,-2.085350,19.796456,-1.892616,4.585266,-6.637573,-2.437592,0
4,-1.285686,19.505561,0.769736,2.372742,-3.517151,-3.608704,-1.807622,19.383457,1.128866,2.784729,...,0.251770,-3.627777,-3.688812,-2.103306,18.825608,-2.163161,-2.956390,-7.122040,-2.307892,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41,0.879869,19.606117,0.538695,2.185822,-4.207611,-4.436493,-1.788469,19.372683,0.597353,2.891541,...,8.102417,-4.795074,-4.676819,-1.696292,19.320010,-4.583699,12.725830,-6.240845,-2.872467,1
42,0.887052,19.597738,0.635661,1.964569,-4.138947,-4.543304,-1.766921,19.348741,1.211466,2.380371,...,-1.644135,-3.955841,-4.653931,-1.264138,19.425355,-3.442862,-4.035950,-7.827759,-2.449036,1
43,0.859518,19.589358,0.549469,2.170563,-4.215240,-4.650116,-1.790863,19.348741,0.652420,2.677917,...,7.251740,-4.634857,-4.451752,-1.848324,18.938135,-1.472434,8.739471,-6.885529,-2.788544,1
44,0.863110,19.583372,0.579397,1.792908,-3.967285,-4.447937,-1.760935,19.346347,0.720655,2.258301,...,-1.857758,-3.963470,-4.722595,-1.668759,19.218257,-2.339135,-0.911713,-7.133484,-2.452850,1


# Scikit Learn - Random Forest

In [23]:
from sklearn.ensemble import RandomForestClassifier
from sklearn_helper import testar_precisao

## IA Gabriel

In [24]:
x_gabriel = df_preparado_gabriel.drop('label', axis=1)
y_gabriel = df_preparado_gabriel['label']

rf_gabriel = RandomForestClassifier()

In [25]:
testar_precisao(rf_gabriel, x_gabriel, y_gabriel)


    Precisão Média:   94.29%
    Desvio Padrão:    0.05
    Variância:        0.0022
    


## IA Joao

In [26]:
x_joao = df_preparado_joao.drop('label', axis=1)
y_joao = df_preparado_joao['label']

rf_joao = RandomForestClassifier()

In [27]:
testar_precisao(rf_joao, x_joao, y_joao)


    Precisão Média:   97.45%
    Desvio Padrão:    0.01
    Variância:        0.0002
    


## Teste Cruzado

In [28]:
# IA treinada com dados de Gabriel testada nos dados de João
testar_precisao(rf_gabriel, x_joao, y_joao)


    Precisão Média:   96.81%
    Desvio Padrão:    0.02
    Variância:        0.0004
    


In [29]:
# IA treinada com dados de João testada nos dados de Gabriel
testar_precisao(rf_joao, x_gabriel, y_gabriel)


    Precisão Média:   92.38%
    Desvio Padrão:    0.05
    Variância:        0.0028
    


## IA com todos os dados

In [30]:
x = df_preparado_gabriel_joao.drop('label', axis=1)
y = df_preparado_gabriel_joao['label']

rf = RandomForestClassifier()

In [31]:
testar_precisao(rf, x, y)


    Precisão Média:   97.35%
    Desvio Padrão:    0.02
    Variância:        0.0003
    
